# Example with Resnet50

## Pre -trained model
Tensorflow Keras provides a pre-trained model to be re-used and extended.     
Here is presented the code, using Resnet50 trained with the imagenet data.    
Reference: https://keras.io/api/applications/#resnet50

In [170]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Dense, Dropout
import numpy as np
import pandas as pd

In [171]:
baseModel = ResNet50(weights="imagenet", include_top=True, input_tensor=Input(shape=(224, 224, 3)))

Now, we use the model to classify our images.     
So, first we define a function 'classify_image' that uses the model to predict the class for a given image.    
Then, we use two images from our cardboard and glass sets, to see which is the assigned class by this pre-trained model. 

Since the model is trained to provide a huge variety of classes, we print the top 3 to better visualize which is the result.  

In [172]:
def classify_image(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return preds # [(class, description, probability)]

In [173]:
img_path = 'dataset-resized/test/cardboard/cardboard116.jpg'
preds = classify_image(img_path, baseModel)
print('Predicted:', decode_predictions(preds, top=3)[0])

FileNotFoundError: [Errno 2] No such file or directory: 'dataset-resized/test/cardboard/cardboard116.jpg'

In [174]:
img_path = 'dataset-resized/training/glass/glass10.jpg'
preds = classify_image(img_path, baseModel)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n03983396', 'pop_bottle', 0.20891704), ('n04070727', 'refrigerator', 0.20570241), ('n03742115', 'medicine_chest', 0.13860318)]


## Adapt to our case
Now, it is time to create our network!. It has as input the output from the pre-trained model. We add some layers to reach to the 6 classes that our dataset has. After our model is ready, we can visualize the final arquitecture of the network.

In [175]:
# Reference: https://www.pyimagesearch.com/2020/04/27/fine-tuning-resnet-with-keras-tensorflow-and-deep-learning/
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(6, activation="softmax")(headModel)
model = tf.keras.Model(inputs=baseModel.input, outputs=headModel)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

We do not have to train all the network again, since the first part is already trained, we can set that those layers do not need training. This will help us with the computational time, as you can see it is a huge network, but our training is going to be focused only in the last part

In [176]:
# loop over all layers in the base model and freeze them so they will not be updated during the training process
for layer in baseModel.layers:\
    layer.trainable = False

## Cross Validation
We need a directory with all images and a labels.csv file with columns: filename, label.

In [181]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

BS = 5
EPOCHS= 10

data_generator = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",
    preprocessing_function=preprocess_input)


train_data = pd.read_csv('labels_csv.csv')
Y = train_data[['label']]

kf = KFold(n_splits = 5)
                         
skf = StratifiedKFold(random_state = 7, shuffle = True)


VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

for train_index, val_index in skf.split(np.zeros(Y.shape[0]),Y):
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]

    train_data_generator = data_generator.flow_from_dataframe(training_data, directory = 'trash-dataset/images',
                               x_col = "filename", y_col = "label",
                               class_mode = "categorical", shuffle = True)
    valid_data_generator  = data_generator.flow_from_dataframe(validation_data, directory = 'trash-dataset/images',
                                x_col = "filename", y_col = "label",
                                class_mode = "categorical", shuffle = True)

    # COMPILE MODEL
    model = tf.keras.Model(inputs=baseModel.input, outputs=headModel)
    model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

    history = model.fit_generator(train_data_generator,
                    validation_steps=len(valid_data_generator) // BS,
                    steps_per_epoch=len(train_data_generator) // BS, epochs=EPOCHS, verbose=False)

    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names,results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])

    tf.keras.backend.clear_session()
    
print('Accuracy:', VALIDATION_ACCURACY)
print('Loss:', VALIDATION_LOSS)

Found 2022 validated image filenames belonging to 6 classes.
Found 506 validated image filenames belonging to 6 classes.
16/16 [==============================] - 31s 2s/step - loss: 0.4982 - accuracy: 0.8340
Found 2022 validated image filenames belonging to 6 classes.
Found 506 validated image filenames belonging to 6 classes.
16/16 [==============================] - 31s 2s/step - loss: 0.3195 - accuracy: 0.8755
Found 2022 validated image filenames belonging to 6 classes.
Found 506 validated image filenames belonging to 6 classes.
16/16 [==============================] - 32s 2s/step - loss: 0.2432 - accuracy: 0.9071
Found 2023 validated image filenames belonging to 6 classes.
Found 505 validated image filenames belonging to 6 classes.
16/16 [==============================] - 33s 2s/step - loss: 0.2228 - accuracy: 0.9030
Found 2023 validated image filenames belonging to 6 classes.
Found 505 validated image filenames belonging to 6 classes.
16/16 [==============================] - 39s 2s

## Accuracy and loss
Show average accuracy and loss of all iterations.

In [182]:
import statistics 

print('Accuracy average:', statistics.mean(VALIDATION_ACCURACY))
print('Loss average:', statistics.mean(VALIDATION_LOSS))

Accuracy average: 0.892033040523529
Loss average: 0.29265692830085754
